# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json 
# Import API key
from api_keys import g_key
!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities = pd.read_csv("../output_data/my_cities.csv", encoding="utf-8")

cities.head()
#cities.drop(["Unnamed: 0"], axis=1)
cities_df = pd.DataFrame (cities)
cities_df.drop(["Unnamed: 0"], axis=1, inplace=True)
cities_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Raudeberg,75,NO,1584476324,80,61.99,5.14,42.8,18.34
1,London,92,GB,1584476174,76,51.51,-0.13,52.0,8.05
2,Cape Town,12,ZA,1584476324,88,-33.93,18.42,68.0,11.41
3,Alanya,40,TR,1584476324,34,36.54,32.00,50.0,23.04
4,Mount Gambier,100,AU,1584476137,33,-37.83,140.77,71.6,14.99


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [26]:
humidity = cities["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = cities[["Lat", "Lng"]]


In [27]:
gmaps.configure(api_key = g_key)
equator_coordinates = (0.00, 0.00)
fig = gmaps.figure(center=equator_coordinates, zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, 
                                 max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions. For example:

  * A max temperature lower than 80 degrees but higher than 70.

  * Wind speed less than 10 mph.

  * Zero cloudiness.

  * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal. 
  

In [28]:
ideal_cities_df = cities.loc[(cities["Max Temp"] > 70) & (cities["Max Temp"] < 80) &
                             (cities["Wind Speed"] < 10) & (cities["Cloudiness"] == 0), :]

ideal_cities_df = ideal_cities_df.dropna(how="any")
ideal_cities_df.reset_index(inplace=True)
del ideal_cities_df["index"]
ideal_cities_df.drop(["Unnamed: 0"], axis=1, inplace=True)
ideal_cities_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Bacolod City,0,PH,1584476335,82,10.67,122.95,72.95,5.95
1,Bilma,0,NE,1584476342,8,18.69,12.92,75.83,7.61
2,Castro,0,BR,1584475919,80,-24.79,-50.01,72.05,5.39
3,Panguna,0,PG,1584476349,89,-6.32,155.48,73.67,0.60
4,Keshabpur,0,BD,1584476355,60,22.92,89.21,72.41,1.39
5,Inhambane,0,MZ,1584476173,73,-23.86,35.38,73.40,6.93
6,Ahmadpur East,0,PK,1584476363,29,29.14,71.26,72.95,5.21
7,Mount Isa,0,AU,1584476364,56,-20.73,139.50,71.60,3.36
8,Raja,0,SS,1584476368,22,8.46,25.68,77.81,3.85
9,Wanparti,0,IN,1584476372,48,16.37,78.07,77.81,5.53


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [29]:
hotel_df = ideal_cities_df
hotel_list = []

for i in range(len(ideal_cities_df)):
    lat = ideal_cities_df.loc[i]['Lat']
    lng = ideal_cities_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotel_list.append(jsn['results'][0]['name'])
    except:
        hotel_list.append("")

ideal_cities_df["Hotel Name"] = hotel_list
ideal_cities_df = ideal_cities_df.dropna(how='any')
ideal_cities_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Bacolod City,0,PH,1584476335,82,10.67,122.95,72.95,5.95,Bacolod
1,Bilma,0,NE,1584476342,8,18.69,12.92,75.83,7.61,Bilma
2,Castro,0,BR,1584475919,80,-24.79,-50.01,72.05,5.39,Castro
3,Panguna,0,PG,1584476349,89,-6.32,155.48,73.67,0.60,Panguna
4,Keshabpur,0,BD,1584476355,60,22.92,89.21,72.41,1.39,Keshabpur
5,Inhambane,0,MZ,1584476173,73,-23.86,35.38,73.40,6.93,Maxixe
6,Ahmadpur East,0,PK,1584476363,29,29.14,71.26,72.95,5.21,Ahmedpur East
7,Mount Isa,0,AU,1584476364,56,-20.73,139.50,71.60,3.36,Mount Isa
8,Raja,0,SS,1584476368,22,8.46,25.68,77.81,3.85,Raga
9,Wanparti,0,IN,1584476372,48,16.37,78.07,77.81,5.53,Wanaparthy


In [30]:
# NOTE: Do not change any of the code in this cell
hotel_df = ideal_cities_df
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = ideal_cities_df[["Lat", "Lng"]]

In [31]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))